# SupplyChain class

In [94]:
import numpy as np
import pandas as pd

from climada.util.api_client import Client
from climada_petals.engine import SupplyChain
from climada.entity import ImpfTropCyclone, ImpactFuncSet
from climada.engine.impact_calc import ImpactCalc

client = Client()

This tutorial shows how to use the `SupplyChain` class of CLIMADA. This class allows assessing indirect impacts via Input-Output modeling. Before diving into this class, it is highly recommended that the user first understands how direct impact is calculated with CLIMADA. This requires getting familiar with the `Exposures`, `Hazard` and `Impact` classes.

This tutorial shows how to set up a global supply chain risk analysis of tropical cyclones that hit the United States.

# 1. Calculate direct economic impacts

The first step is to conduct a direct impact anlysis. To do so, we need to define an exposure, an hazard and a vulnerability. In this tutorial we will load the LitPop exposure for the USA from CLIMADA Data API.

In [115]:
exp_usa = client.get_litpop('USA')

2023-06-01 15:34:15,129 - climada.entity.exposures.base - INFO - Reading /Users/aciullo/climada/data/exposures/litpop/LitPop_150arcsec_USA/v2/LitPop_150arcsec_USA.hdf5


/Users/aciullo/opt/anaconda3/envs/climada_env/lib/python3.9/pickle.py:1717: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  setstate(state)


Then, we load a probabilistic hazard set of tropical cyclones for the USA from the CLIMADA Data API.

In [3]:
tc_usa = client.get_hazard('tropical_cyclone', properties={'country_iso3alpha':'USA', 'climate_scenario':'historical'})

2023-06-01 11:21:33,186 - climada.hazard.base - INFO - Reading /Users/aciullo/climada/data/hazard/tropical_cyclone/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020/v2/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020.hdf5


Then we define vulnerability by loading impact functions for tropical cyclone in the USA:

In [4]:
# Define impact function
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()

And we finally calculate impacts.

In [116]:
# Calculate direct impacts to the USA due to TC
imp_calc = ImpactCalc(exp_usa, impf_set, tc_usa)
direct_impact_usa = imp_calc.impact()

2023-06-01 15:34:19,970 - climada.entity.exposures.base - INFO - No specific impact function column found for hazard TC. Using the anonymous 'impf_' column.
2023-06-01 15:34:19,971 - climada.entity.exposures.base - INFO - Matching 643099 exposures with 503715 centroids.
2023-06-01 15:34:19,977 - climada.util.coordinates - INFO - No exact centroid match found. Reprojecting coordinates to nearest neighbor closer than the threshold = 100
2023-06-01 15:34:20,903 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 129140 coordinates.
2023-06-01 15:34:20,929 - climada.engine.impact_calc - INFO - Calculating impact for 1327884 assets (>0) and 43560 events.


# 2. Calculate indirect economic impacts

## 2.1 Instantiate a `SupplyChain` object by loading the Multi-Regional Input-Output Table of interest.

`SupplyChain` computes indirect economic impacts via Input-Output (IO) modeling. At the core of IO modeling lies an Input-Output Table. `SupplyChain` uses the [pymrio](https://pymrio.readthedocs.io/en/latest/) python package to download, parse and save Multi-Regional Input Output Tables (MRIOTs). In principle, any IO table can be loaded and used, as long as the structure is consistent with those internally supported by `SupplyChain`, which are:
- [EXIOBASE3](https://zenodo.org/record/3583071#.Y_4zZuzMK3I) (1995-2011; 44 countries; 163 industries)​
- [WIOD16](https://www.rug.nl/ggdc/valuechain/wiod/) (2000-2014; 43 countries; 56 industries)​
- [OECD21](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm) (1995-2018; 66 countries; 45 industries)​

These MRIOTs can be downloaded, parsed and saved automatically.

The first step is to instantiate a `SupplyChain` class. This can be done by passing a customized MRIOT or by calling the `from_mriot` class method and use one of the supported MRIOTs.

In [6]:
supchain = SupplyChain.from_mriot(mriot_type='WIOD16', mriot_year=2011)

The instatiated class now has an `mriot` attribute, which is a pymrio `IOSystem` object:

In [10]:
supchain.mriot

As such, one can access several info of the MRIOT incl. regions, sectors, total production, transaction matrix and final demand. Please see the pymrio project on how to make best use of all the provided functions. For example, one can access regions, sectors and IOT data:

In [12]:
# regions
supchain.mriot.get_regions()

Index(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE',
       'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN',
       'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX',
       'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 'SWE',
       'TUR', 'TWN', 'USA', 'ROW'],
      dtype='object', name='region')

In [13]:
# sectors
supchain.mriot.get_sectors()

Index(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel and leather products',
       'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products ',
       'Manufacture of chemicals and chemical products ',
       'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
       'Manufacture of rubber and plastic products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery and equipment',
       

In [14]:
# transaction matrix
supchain.mriot.Z

region                                                                                                                   AUS  \
sector                                                    Crop and animal production, hunting and related service activities   
region sector                                                                                                                  
AUS    Crop and animal production, hunting and related...                                       10954.209508                   
       Forestry and logging                                                                       113.905445                   
       Fishing and aquaculture                                                                     22.267974                   
       Mining and quarrying                                                                       232.437685                   
       Manufacture of food products, beverages and tob...                                        1662.225516                   
...                                                                                                      ...                   
ROW    Education                                                                                    1.305193                   
       Human health and social work activities                                                      0.535337                   
       Other service activities                                                                     1.856064                   
       Activities of households as employers; undiffer...                                           0.027816                   
       Activities of extraterritorial organizations an...                                           0.000000                   

region                                                                          \
sector                                                    Forestry and logging   
region sector                                                                    
AUS    Crop and animal production, hunting and related...           337.318807   
       Forestry and logging                                         127.867760   
       Fishing and aquaculture                                        0.025312   
       Mining and quarrying                                           2.323883   
       Manufacture of food products, beverages and tob...             4.046841   
...                                                                        ...   
ROW    Education                                                      0.022124   
       Human health and social work activities                        0.038688   
       Other service activities                                       0.058495   
       Activities of households as employers; undiffer...             0.000064   
       Activities of extraterritorial organizations an...             0.000000   

region                                                                             \
sector                                                    Fishing and aquaculture   
region sector                                                                       
AUS    Crop and animal production, hunting and related...              166.404066   
       Forestry and logging                                              0.588653   
       Fishing and aquaculture                                          21.871529   
       Mining and quarrying                                              8.353050   
       Manufacture of food products, beverages and tob...               78.987732   
...                                                                           ...   
ROW    Education                                                         0.093113   
       Human health and social work activities                           0.141195   
       Other service activities                                          0.149588   
       Activities of households as emp

In [15]:
# final demand
supchain.mriot.Y

region                                                                                            AUS  \
category                                                  Final consumption expenditure by households   
region sector                                                                                           
AUS    Crop and animal production, hunting and related...                                 7574.242904   
       Forestry and logging                                                                179.441312   
       Fishing and aquaculture                                                            1110.343558   
       Mining and quarrying                                                               3271.376748   
       Manufacture of food products, beverages and tob...                                32371.160467   
...                                                                                               ...   
ROW    Education                                                                            77.238254   
       Human health and social work activities                                              77.839965   
       Other service activities                                                             70.302003   
       Activities of households as employers; undiffer...                                   10.406364   
       Activities of extraterritorial organizations an...                                    0.000000   

region                                                                                                                                          \
category                                                  Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                                                    
AUS    Crop and animal production, hunting and related...                                                  0                                     
       Forestry and logging                                                                                0                                     
       Fishing and aquaculture                                                                             0                                     
       Mining and quarrying                                                                                0                                     
       Manufacture of food products, beverages and tob...                                                  0                                     
...                                                                                                      ...                                     
ROW    Education                                                                                           0                                     
       Human health and social work activities                                                             0                                     
       Other service activities                                                                            0                                     
       Activities of households as employers; undiffer...                                                  0                                     
       Activities of extraterritorial organizations an...                                                  0                                     

region                                                                                                 \
category                                                  Final consumption expenditure by government   
region sector                                                                                           
AUS    Crop and animal production, hunting and related...                                  405.543798   
       Forestry and logging                                     

In [16]:
# total production
supchain.mriot.x

total production
region sector                                                              
AUS    Crop and animal production, hunting and related...      69566.630223
       Forestry and logging                                     2719.293635
       Fishing and aquaculture                                  3113.703184
       Mining and quarrying                                   250207.439237
       Manufacture of food products, beverages and tob...      92033.592425
...                                                                     ...
ROW    Education                                              508147.713351
       Human health and social work activities                399362.733830
       Other service activities                               367396.468789
       Activities of households as employers; undiffer...      22003.013824
       Activities of extraterritorial organizations an...         33.142482

[2464 rows x 1 columns]

## 2.2 Assign exposure and stock impact to MRIOT countries-sectors

After loading the MRIOT, one needs to translate the direct impacts previously calculated, which are defined at an arbitrary spatial resolution, into impacts to sectors and countries defined by the MRIOT. To do this one needs to know what countries and sectors are modeled by the used exposure. The first is straightforward, as exposure contains latitude and longitude information, and even a regional id that often defines the country of interest. The latter needs input from the user, who needs to know/assess what sectors in the MRIOT is the used exposure representative of. For example, assuming the `LitPop`exposure is representative of the service sector, and assuming that sub-sectors at positions 26 to 56 in `WIOD16` do represent this sector, then one can translate spatially disaggregated impacts into country/sector impacts as follows:

In [117]:
impacted_secs = supchain.mriot.get_sectors()[range(26,56)].tolist()
supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs)

Which creates the attributes `self.secs_exp`, `self.secs_imp`, and `self.secs_shock`. The first two show `Exposure` and `Impact` values at the country-sector level. This translation is accomplished assuming that exposure (impact) of an affected sector is proportional to the contribution of this sector to the overall production of all affected sectors. For example, if the total (spatially distributed) exposed value is 100, and there are two affected sectors, A (whose production is 2) and B (whose production is 8); then sector A has an exposure of 20 and sector B has an exposure of 80. The same reasoning is applied to the distributions of direct impacts.

For one easily check that `self.secs_exp`, `self.secs_imp` have the same total values of `Exposure` and `Impact` and that distribution only involved the considered countries and sectors:

In [118]:
# exposure
print(
      exp_usa.gdf.value.sum(),
      supchain.secs_exp.sum().sum(),
      supchain.secs_exp.loc[:, ('USA', impacted_secs)].sum().sum(),
      )

# impact
print(supchain.secs_imp.sum().sum(),
      supchain.secs_imp.loc[:, ('USA', impacted_secs)].sum().sum(),
      direct_impact_usa.imp_mat.sum().sum())

65384554304412.63 65384554304412.64 65384554304412.63
16543899066892.076 16543899066892.076 16543899066892.068


The attribute `self.secs_shock` is proportional to the ratio between `self.secs_imp` and `self.secs_exp`. In particular, `self.secs_shock` contains numbers from 0 and 1 which will be used in the indirect impact calculation to assses how much production loss is experienced by each given sector.

In terms of structure, it is a dataframe with the same columns has the country/sectors in the used MRIOT, and index the event ids of the *non-zero impact events*:

In [119]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

region            USA  \
sector   Construction   
event_id                
10594        0.000352   
10597        0.000634   
10600        0.000694   
10603        0.000032   
10604        0.000071   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
10594                                              0.000352                        
10597                                              0.000634                        
10600                                              0.000694                        
10603                                              0.000032                        
10604                                              0.000071                        

region                                                              \
sector   Wholesale trade, except of motor vehicles and motorcycles   
event_id                                                             
10594                                              0.000352          
10597                                              0.000634          
10600                                              0.000694          
10603                                              0.000032          
10604                                              0.000071          

region                                                           \
sector   Retail trade, except of motor vehicles and motorcycles   
event_id                                                          
10594                                              0.000352       
10597                                              0.000634       
10600                                              0.000694       
10603                                              0.000032       
10604                                              0.000071       

region                                                               \
sector   Land transport and transport via pipelines Water transport   
event_id                                                              
10594                                      0.000352        0.000352   
10597                                      0.000634        0.000634   
10600                                      0.000694        0.000694   
10603                                      0.000032        0.000032   
10604                                      0.000071        0.000071   

region                                                                        \
sector   Air transport Warehousing and support activities for transportation   
event_id                                                                       
10594         0.000352                                           0.000352      
10597         0.000634                                           0.000634      
10600         0.000694                                           0.000694      
10603         0.000032                                           0.000032      
10604         0.000071                                           0.000071      

region                                  \
sector   Postal and courier activities   
event_id                                 
10594                         0.000352   
10597                         0.000634   
10600                         0.000694   
10603                         0.000032   
10604                         0.000071   

region                                              ...  \
sector   Accommodation and food service activities  ...   
event_id                                            ...   
10594                                     0.000352  ...   
10597                                     0.000634  ...   
10600                                     0.000694  ...   
10603                                     0.000032  ...   
10604                                     0.000071  ...   

region  

By default, `self.secs_shock` is exactly the ratio between `Impact` and `Exposure`, which results in the same shock across sectors for a given event:

In [120]:
# let's try the first three events
for event_id in supchain.secs_shock.index[:3]:
    imp_event = direct_impact_usa.at_event[direct_impact_usa.event_id == event_id][0]
    print(imp_event / exp_usa.gdf.value.sum(), supchain.secs_shock.loc[event_id, ('USA', impacted_secs)].values[0])

0.0003524059371639781 0.0003524059371639781
0.0006340759642834555 0.0006340759642834555
0.0006936414424383894 0.0006936414424383894


This practically means that the fraction of production losses are assumed to be equal to the fractions of stock losses, since `Impact` and `Exposure` typically refer to stocks in CLIMADA. However, since depending on the sector one can reasonably expect production losses to be proportionally higher or lower than stock losses, a `shock_factor` argument can also be passed to define - for each sector - how much should production shocks be higher/lower than the stock shocks (i.e., the mere `Impact` / `Exposure` ratio):

In [163]:
shock_factor = pd.DataFrame(np.repeat(1, supchain.mriot.x.shape[0]), 
                            index=supchain.mriot.x.index,
                            columns=['shock'])

# randomly generated
shock_facs_service_USA = np.array([
                                   0.38324804, 1.15930626, 0.73846477, 0.5430206 , 0.54147014,
                                   0.28362671, 0.53829353, 1.95367016, 1.33675622, 0.42285787,
                                   0.86974667, 1.4685637 , 1.24804793, 0.56915521, 0.43723048,
                                   0.23372398, 0.69268485, 0.74130451, 0.74739106, 1.18719852,
                                   1.02203697, 1.0412411 , 0.09315484, 1.23612412, 0.55947349,
                                   0.8608431, 0.58983156, 1.13137055, 0.93014364, 0.39092134
                                   ])

shock_factor.loc[('USA', impacted_secs), :] = shock_facs_service_USA

supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs, shock_factor.values.flatten())

In [164]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

region            USA  \
sector   Construction   
event_id                
10594        0.000135   
10597        0.000243   
10600        0.000266   
10603        0.000012   
10604        0.000027   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
10594                                              0.000409                        
10597                                              0.000735                        
10600                                              0.000804                        
10603                                              0.000037                        
10604                                              0.000082                        

region                                                              \
sector   Wholesale trade, except of motor vehicles and motorcycles   
event_id                                                             
10594                                              0.000260          
10597                                              0.000468          
10600                                              0.000512          
10603                                              0.000024          
10604                                              0.000052          

region                                                           \
sector   Retail trade, except of motor vehicles and motorcycles   
event_id                                                          
10594                                              0.000191       
10597                                              0.000344       
10600                                              0.000377       
10603                                              0.000017       
10604                                              0.000039       

region                                                               \
sector   Land transport and transport via pipelines Water transport   
event_id                                                              
10594                                      0.000191        0.000100   
10597                                      0.000343        0.000180   
10600                                      0.000376        0.000197   
10603                                      0.000017        0.000009   
10604                                      0.000038        0.000020   

region                                                                        \
sector   Air transport Warehousing and support activities for transportation   
event_id                                                                       
10594         0.000190                                           0.000688      
10597         0.000341                                           0.001239      
10600         0.000373                                           0.001355      
10603         0.000017                                           0.000062      
10604         0.000038                                           0.000139      

region                                  \
sector   Postal and courier activities   
event_id                                 
10594                         0.000471   
10597                         0.000848   
10600                         0.000927   
10603                         0.000043   
10604                         0.000095   

region                                              ...  \
sector   Accommodation and food service activities  ...   
event_id                                            ...   
10594                                     0.000149  ...   
10597                                     0.000268  ...   
10600                                     0.000293  ...   
10603                                     0.000013  ...   
10604                                     0.000030  ...   

region  

Although the default values for the shock factors are "true" only in the (uncommon) case in which CLIMADA's direct impacts already express production losses, a proper assignment of these factors require extensive expert knowledge on how a sector's production responds to the sector's stock losses. Therefore, it is recommended to change these values only when detailed knowledge about the relationship between stock and production losses is available.

## 2.3 Calculate the propagation of production losses

In order to calculate how these local production losses propagate through the supply chain, one can use the `calc_indirect_production_impacts` method. This calculation can be executed following the `ghosh` or the `leontief` methods.

In [ ]:
supchain.calc_impacts(io_approach='ghosh')
supchain.calc_impacts(io_approach='leontief')
supchain.calc_impacts(io_approach='eeioa')

In [ ]:
supchain.supchain_imp['ghosh']

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
10594                                       131259.878649                   
10597                                       236172.905587                   
10600                                       258359.130647                   
10603                                        11878.869010                   
10604                                        26415.675130                   
...                                                   ...                   
43408                                         3235.122057                   
43409                                          615.189544                   
43410                                          376.022099                   
43413                                            0.004213                   
43415                                          106.767746                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
10594           4699.507608             6713.290457        454169.166123   
10597           8455.716842            12079.070388        817176.220909   
10600           9250.051977            13213.785538        893942.247501   
10603            425.300068              607.545114         41101.790497   
10604            945.762464             1351.030502         91400.245603   
...                     ...                     ...                  ...   
43408            115.827326              165.460415         11193.768439   
43409             22.025679               31.463888          2128.602623   
43410             13.462748               19.231662          1301.065071   
43413              0.000151                0.000215             0.014577   
43415              3.822614                5.460640           369.424524   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
10594                                       196811.856212             
10597                                       354119.083562             
10600                                       387385.243650             
10603                                        17811.248065             
10604                                        39607.823113             
...                                                   ...             
43408                                         4850.761586             
43409                                          922.418924             
43410                                          563.809811             
43413                                            0.006317             
43415                                          160.088204             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
10594                                        20135.581314              
10597                                        36229.492161              
10600                                        39632.912485              
10603                                         1822.247097              
10604                                         4052.228145              
...                                                   ...              
43408                                          496.275510              
43409                                           94.371557              
43410                                           57.682695              
43413                                            0.000646              
43415                                           16.378429              

region                                                                                                                                  \
sector Man

In [ ]:
supchain.supchain_imp['leontief']

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
10594                                        76998.347241                   
10597                                       138541.369842                   
10600                                       151556.029604                   
10603                                         6968.262429                   
10604                                        15495.697141                   
...                                                   ...                   
43408                                         1897.754699                   
43409                                          360.876290                   
43410                                          220.578295                   
43413                                            0.002471                   
43415                                           62.631019                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
10594           7091.357632             3004.296527         1.500951e+06   
10597          12759.318033             5405.562212         2.700627e+06   
10600          13957.936057             5913.363983         2.954326e+06   
10603            641.759762              271.885402         1.358344e+05   
10604           1427.115440              604.606083         3.020621e+05   
...                     ...                     ...                  ...   
43408            174.778521               74.045977         3.699348e+04   
43409             33.235815               14.080554         7.034666e+03   
43410             20.314716                8.606452         4.299797e+03   
43413              0.000228                0.000096         4.817323e-02   
43415              5.768162                2.443717         1.220885e+03   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
10594                                        91266.654709             
10597                                       164214.010006             
10600                                       179640.373055             
10603                                         8259.527949             
10604                                        18367.153206             
...                                                   ...             
43408                                         2249.421306             
43409                                          427.749075             
43410                                          261.452924             
43413                                            0.002929             
43415                                           74.236965             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
10594                                        22175.207650              
10597                                        39899.345304              
10600                                        43647.513843              
10603                                         2006.830950              
10604                                         4462.697111              
...                                                   ...              
43408                                          546.545556              
43409                                          103.930889              
43410                                           63.525643              
43413                                            0.000712              
43415                                           18.037476              

region                                                                                                                                  \
sector Man

In [ ]:
supchain.supchain_imp['eeioa']

region                                                                AUS  \
sector Crop and animal production, hunting and related service activities   
10594                                       218164.147240                   
10597                                       392537.773758                   
10600                                       429413.008753                   
10603                                        19743.606001                   
10604                                        43904.910608                   
...                                                   ...                   
43408                                         5377.024968                   
43409                                         1022.492962                   
43410                                          624.978032                   
43413                                            0.007002                   
43415                                          177.456314                   

region                                                                    \
sector Forestry and logging Fishing and aquaculture Mining and quarrying   
10594           7820.004142            11075.503276         7.575065e+05   
10597          14070.355077            19927.900409         1.362964e+06   
10600          15392.132712            21799.939381         1.491002e+06   
10603            707.701438             1002.320389         6.855347e+04   
10604           1573.753466             2228.913354         1.524460e+05   
...                     ...                     ...                  ...   
43408            192.737249              272.974540         1.867003e+04   
43409             36.650840               51.908732         3.550286e+03   
43410             22.402081               31.728157         2.170040e+03   
43413              0.000251                0.000355         2.431228e-02   
43415              6.360849                9.008895         6.161614e+02   

region                                                               \
sector Manufacture of food products, beverages and tobacco products   
10594                                       327971.923671             
10597                                       590112.401152             
10600                                       645547.915695             
10603                                        29681.084277             
10604                                        66003.411527             
...                                                   ...             
43408                                         8083.423627             
43409                                         1537.140672             
43410                                          939.545980             
43413                                            0.010526             
43415                                          266.774763             

region                                                                \
sector Manufacture of textiles, wearing apparel and leather products   
10594                                        33402.645967              
10597                                        60100.618967              
10600                                        65746.507327              
10603                                         3022.901287              
10604                                         6722.186958              
...                                                   ...              
43408                                          823.264792              
43409                                          156.551711              
43410                                           95.689050              
43413                                            0.001072              
43415                                           27.169957              

region                                                                                                                                  \
sector Man

This creates the following outputs:

In [ ]:
supchain.indir_prod_impt_mat

which is a `pandas.DataFrame` with indexes equal to the hazard events id and columns that represent the country-sectors in the MRIOT. This local production loss is obviously located only in the USA and only affect the sectors that are representative of the used exposure:

This time we don't only have impacts to the USA, but also to other countries, e.g. Switzerland.

In [ ]:
supchain.indir_prod_impt_mat.loc[:,('CHE', slice(None))].sum().sum()

## Calculate expected production losses

In [ ]:
supchain.calc_production_eai(direct_impact_usa.frequency)

In [ ]:
supchain.dir_prod_impt_eai.loc[('USA', impacted_secs)]

In [ ]:
supchain.indir_prod_impt_eai.loc[('CHE', slice(None))]

In [ ]:
supchain.tot_prod_impt_eai.loc[('CHE', slice(None))]